In [1]:
trash = True
# modify_datset = 'no'
# modify_datset = 'top'
modify_datset = 'coeff'

from google.colab import drive
drive.mount('/content/drive')
import sys
datasetpath = '/content/drive/MyDrive/bio/dataset'
databasepath = '/content/drive/MyDrive/bio/database'
utilspath = '/content/drive/MyDrive/bio/utils'
sys.path.append(datasetpath)
sys.path.append(databasepath)
sys.path.append(utilspath)

! pip3 install selenium
! pip3 install wikipedia
! pip3 install transformers
! pip3 install spacy
! pip3 install transformer
! pip3 install neuralcoref
! python -m spacy download en_core_web_md
! pip3 install bert-extractive-summarizer
! pip3 install unidecode
! pip3 install sentencepiece
! pip3 install sentence_transformers

from persons import Person, Article, DataBase, correct_str
from shortnews import ShortNews, ShortNewsItem, create_news, create_bio, cut_news
import pickle, random
from random import shuffle
from utils import get_tokenizer
from tqdm.notebook import tqdm
import nltk, numpy as np
nltk.download('punkt')
import sentence_transformers


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from summarizer import Summarizer
model = Summarizer()
tokenizer = get_tokenizer('gpt2-large')

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
databasename = '/database.pickle'
datasetname = '/dataset.pickle'
if trash:
  databasename = '/database_trash.pickle'
  datasetname = '/dataset_trash.pickle'

with open(databasepath + databasename, 'rb') as f:
  database = pickle.load(f)

# shortnews = ShortNews()
# with open(datasetpath + datasetname, 'wb') as f:
#  pickle.dump(shortnews, f)
with open(datasetpath + datasetname, 'rb') as f:
  shortnews = pickle.load(f)

blacklist = ['bbc.co.uk', 'bbc.com', 'Follow us', 'follow us',
             'Follow BBC', 'follow BBC', 'Follow @', 'follow @',
             'BBC Monitoring reports and analyses',
             'click here', 'download the BBC Newsbeat',
             'For iPhone go here', 'For Android go here', 'For iOS go here']

for person in tqdm(database.persons):
    print(person.names[0])
    i = shortnews.find(person.names[0])

    if i is None:
      print('Here')
      news = create_news(person.articles, blacklist, model, correct_str)
      bio = create_bio(person.summary, correct_str, n_sent=3)
      news = cut_news(tokenizer, bio, news, 1000)
      shortnews.items.append(
          ShortNewsItem(name=person.names[0], news=news, bio=bio))

    if modify_datset == 'top' or modify_datset == 'coeff':
      print('creating new bio')
      i = shortnews.find(person.names[0])
      bio = person.summary
      new = shortnews.items[i].news

      bios = nltk.tokenize.sent_tokenize(correct_str(bio, first_ignore=False))
      news = nltk.tokenize.sent_tokenize(correct_str(new, first_ignore=False))
      model_cos = sentence_transformers.SentenceTransformer(
          'paraphrase-distilroberta-base-v1')
      embeddingsb = model_cos.encode(bios, convert_to_tensor=True)
      embeddingsn = model_cos.encode(news, convert_to_tensor=True)
      cosine_scores = np.array(sentence_transformers.util.pytorch_cos_sim(
          embeddingsb, embeddingsn).cpu()).max(axis=1)

      if modify_datset == 'top':
        cosine_scores = cosine_scores[1:].argsort()[-3:][::-1] + 1
        cosine_scores.sort()
        cosine_scores = np.insert(cosine_scores, 0, 0, axis=0)
        
        bio = ''
        for j in cosine_scores.tolist():
          if bio != '' and bio[-1] != ' ' and bio[-1] != '\n':
            bio += ' '
            bio += bios[j]    
        news = new

      if modify_datset == 'coeff':
        ids = np.array(sentence_transformers.util.pytorch_cos_sim(
            embeddingsb, embeddingsn).cpu()).max(axis=1) > 0.5
        #print(ids)
        good_bios = []
        for k, biography in enumerate(bios):
          if ids[k]:
             good_bios.append(biography)
        news_idx = np.array(sentence_transformers.util.pytorch_cos_sim(
            embeddingsb, embeddingsn).cpu()).argmax(axis=1)[ids]

        # for k in range(len(good_bios)):
        #   print(good_bios[k])
        #   print(news[int(news_idx[k])])

        if len(good_bios) == 0 or good_bios[0] != bios[0]:
          good_bios = [bios[0]] + good_bios

        bio = ''
        for bio_str in good_bios:
          if bio != '' and bio[-1] != ' ' and bio[-1] != '\n':
            bio += ' '
          bio += bio_str   
        news = new
      
      news = cut_news(tokenizer, bio, news, 1000)
      shortnews.items[i].bio = bio
      shortnews.items[i].news = news

    #random.Random(42).shuffle(shortnews.items)
    with open(datasetpath + datasetname, 'wb') as f:
      pickle.dump(shortnews, f)
    print()

Jang Jin
creating new bio
Length before cut: 705
is a South Korean film director, theatre director, playwright, screenwriter, film producer, actor and TV personality.
As part of North Korea's propaganda machine, Jang Jin-sung spent his career writing eulogies of Kim Jong-il, before growing disillusioned and fleeing to South Korea in 2004. But when I met him I felt he was much more individualistic, even a bit selfish - and I was disappointed. After we met, Kim Jong-il asked his aides to take care of me and afterwards I received special treatment - the kind of benefits unavailable to normal citizens. Once you have met him in person, you can't even be prosecuted in court without a special signing off. For privileged, higher class, citizens there are three types of rations - a daily ration, a three day ration and a weekly ration. The daily ration is the highest, I received a weekly ration which was still very lavish. I saw that most people were living an entirely different life from the el